In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2

In [23]:
class AdaBoost:
    def __init__(self) -> None:
        self.list_of_weak_learners = []
        self.alphas = []

    def fit(self, X, y, number_of_weak_learners):
        n = len(X)
        w = np.ones(n) / n
        for t in range(number_of_weak_learners):
            weak_learner = self._get_weak_learner(X, y, w)
            self.list_of_weak_learners.append(weak_learner)
            y_pred = weak_learner.predict(X)
            epsilon = np.sum(w * (y_pred != y))
            alpha = 0.5 * np.log((1 - epsilon) / epsilon)
            self.alphas.append(alpha)
            w = w * np.exp(-alpha * y * y_pred)
            w /= np.sum(w)

    def predict(self, X):
        predictions = np.zeros(len(X))
        for i, weak_learner in enumerate(self.list_of_weak_learners):
            predictions += self.alphas[i] * weak_learner.predict(X)
        return np.sign(predictions)
    
    def _get_weak_learner(self, X, y, w):
        

In [21]:
def generate_hard_data(number_of_sample, number_of_features):
    X = np.random.randn(number_of_sample, number_of_features)
    median_of_chi2 = chi2.median(number_of_features)
    sum_of_squares = np.sum(X**2, axis=1, keepdims=True)
    y = np.where(sum_of_squares > median_of_chi2, 1, -1)
    return X, y